In [215]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

pd.set_option('display.max_columns', None)


In [2]:
solar = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/elecsim/data/processed/capacity_factor/Solar/ninja_pv_country_GB_merra-2_corrected.csv')
wind = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/elecsim/data/processed/capacity_factor/Wind/ninja_wind_country_GB_current-merra-2_corrected.csv')
demand = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/elecsim/data/processed/electricity_demand/uk_all_year_demand.csv')
wind.head()

,Unnamed: 0,time,national,offshore,onshore
0,0,01/01/1980 00:00,0.2024,0.2858,0.1496
1,1,01/01/1980 01:00,0.2016,0.2827,0.1502
2,2,01/01/1980 02:00,0.2015,0.2852,0.1485
3,3,01/01/1980 03:00,0.2032,0.2920,0.1470
4,4,01/01/1980 04:00,0.2090,0.3039,0.1489


## Data Munging and Feature Engineering

In [3]:
onshore = wind[['time','onshore']]
offshore = wind[['time','offshore']]
offshore.head()

,time,offshore
0,01/01/1980 00:00,0.2858
1,01/01/1980 01:00,0.2827
2,01/01/1980 02:00,0.2852
3,01/01/1980 03:00,0.2920
4,01/01/1980 04:00,0.3039


In [4]:
solar = solar.rename(columns={'pv':'value'})
offshore = offshore.rename(columns={'offshore':'value'})
onshore = onshore.rename(columns={'onshore':'value'})
demand = demand.rename(columns={'timestamp':'time','demand':'value'})
onshore.head()

,time,value
0,01/01/1980 00:00,0.1496
1,01/01/1980 01:00,0.1502
2,01/01/1980 02:00,0.1485
3,01/01/1980 03:00,0.1470
4,01/01/1980 04:00,0.1489


In [5]:
def to_datetime(dat, col_to_datetime):
    dat[col_to_datetime] = pd.to_datetime(dat[col_to_datetime])
    return dat


In [6]:
offshore = to_datetime(offshore, 'time')
onshore = to_datetime(onshore, 'time')
solar = to_datetime(solar, 'time')
demand = to_datetime(demand, 'time')

In [7]:
def get_hour(dat, datetime):
    hour_series = dat[datetime].dt.hour
    return hour_series
    


In [8]:
offshore['hour'] = get_hour(offshore, 'time')
onshore['hour'] = get_hour(onshore, 'time')
solar['hour'] = get_hour(solar, 'time')
demand['hour'] = get_hour(demand, 'time')
demand.head()

,time,value,hour
0,2011-05-27 16:00:00,38649.090909,16
1,2011-05-27 17:00:00,37497.583333,17
2,2011-05-27 18:00:00,36146.333333,18
3,2011-05-27 19:00:00,34741.166667,19
4,2011-05-27 20:00:00,34722.666667,20


In [9]:
def get_month(dat, datetime):
    month_series = dat[datetime].dt.month
    return month_series

In [10]:
offshore['month'] = get_month(offshore, 'time')
onshore['month'] = get_month(onshore, 'time')
solar['month'] = get_month(solar, 'time')
demand['month'] = get_month(demand, 'time')

demand.head()

,time,value,hour,month
0,2011-05-27 16:00:00,38649.090909,16,5
1,2011-05-27 17:00:00,37497.583333,17,5
2,2011-05-27 18:00:00,36146.333333,18,5
3,2011-05-27 19:00:00,34741.166667,19,5
4,2011-05-27 20:00:00,34722.666667,20,5


In [11]:
def get_dayofweek(dat, datetime):
    dayofweek_series = dat[datetime].dt.dayofweek
    return dayofweek_series

In [12]:
offshore['dayofweek'] = get_dayofweek(offshore, 'time')
onshore['dayofweek'] = get_dayofweek(onshore, 'time')
solar['dayofweek'] = get_dayofweek(solar, 'time')
demand['dayofweek'] = get_dayofweek(demand, 'time')

solar.head()

,time,value,hour,month,dayofweek
0,1985-01-01 00:00:00,0.0,0,1,1
1,1985-01-01 01:00:00,0.0,1,1,1
2,1985-01-01 02:00:00,0.0,2,1,1
3,1985-01-01 03:00:00,0.0,3,1,1
4,1985-01-01 04:00:00,0.0,4,1,1


In [13]:
def get_dayofmonth(dat, datetime):
    dayofmonth_series = dat[datetime].dt.day
    return dayofmonth_series

In [14]:
offshore['dayofmonth'] = get_dayofmonth(offshore, 'time')
onshore['dayofmonth'] = get_dayofmonth(onshore, 'time')
solar['dayofmonth'] = get_dayofmonth(solar, 'time')
demand['dayofmonth'] = get_dayofmonth(demand, 'time')

solar.head()

,time,value,hour,month,dayofweek,dayofmonth
0,1985-01-01 00:00:00,0.0,0,1,1,1
1,1985-01-01 01:00:00,0.0,1,1,1,1
2,1985-01-01 02:00:00,0.0,2,1,1,1
3,1985-01-01 03:00:00,0.0,3,1,1,1
4,1985-01-01 04:00:00,0.0,4,1,1,1


In [15]:
def get_lags(dat, value):
    previous_three_hours = [1,2,3,]
    previous_day = [24,25,26,27]
    previous_week = [168,169,170,171]
    lags_required = previous_three_hours + previous_day + previous_week

    new_cols = dat.assign(**{f'value-{lag}': dat[value].shift(lag) for lag in lags_required})
    return new_cols

In [16]:
offshore = get_lags(offshore, 'value')
onshore = get_lags(onshore, 'value')
solar = get_lags(solar, 'value')
demand = get_lags(demand, 'value')
solar.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171
0,1985-01-01 00:00:00,0.0,0,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1985-01-01 01:00:00,0.0,1,1,1,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-01-01 02:00:00,0.0,2,1,1,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1985-01-01 03:00:00,0.0,3,1,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1985-01-01 04:00:00,0.0,4,1,1,1,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
offshore = offshore.dropna()
onshore = onshore.dropna()
solar = solar.dropna()
demand = demand.dropna()
demand

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171
171,2011-06-03 19:00:00,33880.250000,19,6,4,3,35690.250000,36978.333333,38135.416667,35907.000000,37420.250000,38815.583333,39945.250000,34741.166667,36146.333333,37497.583333,38649.090909
172,2011-06-03 20:00:00,33181.000000,20,6,4,3,33880.250000,35690.250000,36978.333333,35405.833333,35907.000000,37420.250000,38815.583333,34722.666667,34741.166667,36146.333333,37497.583333
173,2011-06-03 21:00:00,33132.083333,21,6,4,3,33181.000000,33880.250000,35690.250000,35090.250000,35405.833333,35907.000000,37420.250000,33434.833333,34722.666667,34741.166667,36146.333333
174,2011-06-03 22:00:00,30041.916667,22,6,4,3,33132.083333,33181.000000,33880.250000,31313.083333,35090.250000,35405.833333,35907.000000,29708.750000,33434.833333,34722.666667,34741.166667
175,2011-06-03 23:00:00,27883.666667,23,6,4,3,30041.916667,33132.083333,33181.000000,29467.250000,31313.083333,35090.250000,35405.833333,27410.000000,29708.750000,33434.833333,34722.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66502,2018-12-27 14:00:00,36784.750000,14,12,3,27,36682.000000,36814.166667,36714.583333,32605.500000,32773.250000,32548.916667,31786.916667,39188.750000,38287.500000,38239.750000,38465.000000
66503,2018-12-27 15:00:00,37835.750000,15,12,3,27,36784.750000,36682.000000,36814.166667,33133.333333,32605.500000,32773.250000,32548.916667,40193.583333,39188.750000,38287.500000,38239.750000
66504,2018-12-27 16:00:00,40393.583333,16,12,3,27,37835.750000,36784.750000,36682.000000,35031.833333,33133.333333,32605.500000,32773.250000,42569.583333,40193.583333,39188.750000,38287.500000
66505,2018-12-27 17:00:00,42150.583333,17,12,3,27,40393.583333,37835.750000,36784.750000,36038.833333,35031.833333,33133.333333,32605.500000,44252.916667,42569.583333,40193.583333,39188.750000


In [18]:
from pandas.tseries.holiday import *
from pandas.tseries.offsets import CustomBusinessDay

class EnglandAndWalesHolidayCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('New Years Day', month=1, day=1, observance=next_monday),
        GoodFriday,
        EasterMonday,
        Holiday('Early May bank holiday',
                month=5, day=1, offset=DateOffset(weekday=MO(1))),
        Holiday('Spring bank holiday',
                month=5, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Summer bank holiday',
                month=8, day=31, offset=DateOffset(weekday=MO(-1))),
        Holiday('Christmas Day', month=12, day=25, observance=next_monday),
        Holiday('Boxing Day',
                month=12, day=26, observance=next_monday_or_tuesday)
    ]

def check_if_holiday(dat, datetime):
    cal = EnglandAndWalesHolidayCalendar()
    
    holidays = cal.holidays(start=dat[datetime].min(), end=dat[datetime].max())

    dat['holiday'] = dat['time'].dt.date.astype('datetime64').isin(holidays)
    return dat

In [19]:
solar = check_if_holiday(solar, 'time')
offshore = check_if_holiday(offshore, 'time')
onshore = check_if_holiday(onshore, 'time')
demand = check_if_holiday(demand, 'time')

In [20]:
onshore.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,1980-08-01 03:00:00,0.2022,3,8,4,1,0.1860,0.1758,0.1756,0.1765,0.1722,0.1702,0.1805,0.1470,0.1485,0.1502,0.1496,False
172,1980-08-01 04:00:00,0.2165,4,8,4,1,0.2022,0.1860,0.1758,0.1778,0.1765,0.1722,0.1702,0.1489,0.1470,0.1485,0.1502,False
173,1980-08-01 05:00:00,0.2146,5,8,4,1,0.2165,0.2022,0.1860,0.1751,0.1778,0.1765,0.1722,0.1483,0.1489,0.1470,0.1485,False
174,1980-08-01 06:00:00,0.2028,6,8,4,1,0.2146,0.2165,0.2022,0.1683,0.1751,0.1778,0.1765,0.1475,0.1483,0.1489,0.1470,False
175,1980-08-01 07:00:00,0.1967,7,8,4,1,0.2028,0.2146,0.2165,0.1654,0.1683,0.1751,0.1778,0.1486,0.1475,0.1483,0.1489,False


In [21]:
offshore.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,1980-08-01 03:00:00,0.2285,3,8,4,1,0.2170,0.2067,0.2095,0.2822,0.3081,0.3314,0.3443,0.2920,0.2852,0.2827,0.2858,False
172,1980-08-01 04:00:00,0.2347,4,8,4,1,0.2285,0.2170,0.2067,0.2617,0.2822,0.3081,0.3314,0.3039,0.2920,0.2852,0.2827,False
173,1980-08-01 05:00:00,0.2318,5,8,4,1,0.2347,0.2285,0.2170,0.2441,0.2617,0.2822,0.3081,0.3078,0.3039,0.2920,0.2852,False
174,1980-08-01 06:00:00,0.2265,6,8,4,1,0.2318,0.2347,0.2285,0.2338,0.2441,0.2617,0.2822,0.3119,0.3078,0.3039,0.2920,False
175,1980-08-01 07:00:00,0.2206,7,8,4,1,0.2265,0.2318,0.2347,0.2294,0.2338,0.2441,0.2617,0.3148,0.3119,0.3078,0.3039,False


In [22]:
solar.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,1985-08-01 03:00:00,0.0,3,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
172,1985-08-01 04:00:00,0.0,4,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
173,1985-08-01 05:00:00,0.0,5,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
174,1985-08-01 06:00:00,0.0,6,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False
175,1985-08-01 07:00:00,0.0,7,8,3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False


In [23]:
demand.head()

,time,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
171,2011-06-03 19:00:00,33880.250000,19,6,4,3,35690.250000,36978.333333,38135.416667,35907.000000,37420.250000,38815.583333,39945.250000,34741.166667,36146.333333,37497.583333,38649.090909,False
172,2011-06-03 20:00:00,33181.000000,20,6,4,3,33880.250000,35690.250000,36978.333333,35405.833333,35907.000000,37420.250000,38815.583333,34722.666667,34741.166667,36146.333333,37497.583333,False
173,2011-06-03 21:00:00,33132.083333,21,6,4,3,33181.000000,33880.250000,35690.250000,35090.250000,35405.833333,35907.000000,37420.250000,33434.833333,34722.666667,34741.166667,36146.333333,False
174,2011-06-03 22:00:00,30041.916667,22,6,4,3,33132.083333,33181.000000,33880.250000,31313.083333,35090.250000,35405.833333,35907.000000,29708.750000,33434.833333,34722.666667,34741.166667,False
175,2011-06-03 23:00:00,27883.666667,23,6,4,3,30041.916667,33132.083333,33181.000000,29467.250000,31313.083333,35090.250000,35405.833333,27410.000000,29708.750000,33434.833333,34722.666667,False


In [24]:
demand.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/demand.csv')
solar.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/solar.csv')
offshore.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/offshore.csv')
onshore.to_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/onshore.csv')

In [46]:
demand = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/demand.csv')
solar = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/solar.csv')
offshore = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/offshore.csv')
onshore = pd.read_csv('/Users/b1017579/Documents/PhD/Projects/10-ELECSIM/run/market-forecasting-comparison/data/onshore.csv')

In [47]:
demand = demand.drop(columns=['time','Unnamed: 0'])
solar = solar.drop(columns=['time','Unnamed: 0'])
onshore = onshore.drop(columns=['time','Unnamed: 0'])
offshore = offshore.drop(columns=['time','Unnamed: 0'])
demand.head()

,value,hour,month,dayofweek,dayofmonth,value-1,value-2,value-3,value-24,value-25,value-26,value-27,value-168,value-169,value-170,value-171,holiday
0,33880.250000,19,6,4,3,35690.250000,36978.333333,38135.416667,35907.000000,37420.250000,38815.583333,39945.250000,34741.166667,36146.333333,37497.583333,38649.090909,False
1,33181.000000,20,6,4,3,33880.250000,35690.250000,36978.333333,35405.833333,35907.000000,37420.250000,38815.583333,34722.666667,34741.166667,36146.333333,37497.583333,False
2,33132.083333,21,6,4,3,33181.000000,33880.250000,35690.250000,35090.250000,35405.833333,35907.000000,37420.250000,33434.833333,34722.666667,34741.166667,36146.333333,False
3,30041.916667,22,6,4,3,33132.083333,33181.000000,33880.250000,31313.083333,35090.250000,35405.833333,35907.000000,29708.750000,33434.833333,34722.666667,34741.166667,False
4,27883.666667,23,6,4,3,30041.916667,33132.083333,33181.000000,29467.250000,31313.083333,35090.250000,35405.833333,27410.000000,29708.750000,33434.833333,34722.666667,False


## Machine Learning and Prediction 

In [48]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV

from sklearn import datasets

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.svm import SVC

In [245]:
class EstimatorSelectionHelper:

    def __init__(self, models, params, scoring=None):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError("Some estimators are missing parameters: %s" % missing_params)
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}
        self.scoring = scoring

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, refit=False):
        for key in self.keys:
            print("Running GridSearchCV for %s." % key)
            model = self.models[key]
            params = self.params[key]
            tscv = TimeSeriesSplit(n_splits=cv)

            gs = GridSearchCV(model, params, cv=tscv, n_jobs=n_jobs,
                              verbose=verbose, scoring=self.scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X,y)
            self.grid_searches[key] = gs    

    def score_summary(self):
        all_results = []
        for k in self.grid_searches:
            results = pd.DataFrame(self.grid_searches[k].cv_results_)
            results['estimator'] = k
            all_results.append(results)
        results_df = pd.concat(all_results)
        results_df = results_df.loc[:,~results_df.columns.str.contains('train')] 

        return results_df

In [246]:


models = {
    'ExtraTreesRegressor': ExtraTreesRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
# #     'SVR': SVR()
}

params = {
    'ExtraTreesRegressor': { 'n_estimators': [16, 32] },
    'RandomForestRegressor': { 'n_estimators': [16, 32] },
    'AdaBoostRegressor':  { 'n_estimators': [16, 32] },
    'GradientBoostingRegressor': { 'n_estimators': [16, 32], 'learning_rate': [0.8, 1.0] },
# #     'SVC': [
# #         {'kernel': ['linear'], 'C': [1, 10]},
# #         {'kernel': ['rbf'], 'C': [1, 10], 'gamma': [0.001, 0.0001]},
# #     ]
}

In [247]:
X = demand.drop(columns='value')
y = demand.value

In [258]:
helper1 = EstimatorSelectionHelper(models, params, scoring=['neg_mean_absolute_error','neg_mean_squared_error','neg_median_absolute_error','r2'])
helper1.fit(X, y, n_jobs=-1, cv=2)

Running GridSearchCV for ExtraTreesRegressor.
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   10.7s remaining:   10.7s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   16.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Running GridSearchCV for RandomForestRegressor.
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:   18.5s remaining:   18.5s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   29.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Running GridSearchCV for AdaBoostRegressor.
Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    6.5s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    9.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Running GridSearchCV for GradientBoostingRegressor.
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    9.1s finished


In [261]:
res = helper1.score_summary()
res.sort_values('mean_test_neg_mean_squared_error', ascending = False)


/Users/b1017579/.pyenv/versions/3.6.6/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,estimator,mean_fit_time,mean_score_time,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_neg_median_absolute_error,mean_test_r2,param_learning_rate,param_n_estimators,params,rank_test_neg_mean_absolute_error,rank_test_neg_mean_squared_error,rank_test_neg_median_absolute_error,rank_test_r2,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error,split0_test_neg_median_absolute_error,split0_test_r2,split1_test_neg_mean_absolute_error,split1_test_neg_mean_squared_error,split1_test_neg_median_absolute_error,split1_test_r2,std_fit_time,std_score_time,std_test_neg_mean_absolute_error,std_test_neg_mean_squared_error,std_test_neg_median_absolute_error,std_test_r2
1,ExtraTreesRegressor,7.872368,0.981318,-537.203726,-8.077965e+05,-371.479167,0.982563,NaN,32,{'n_estimators': 32},1,1,1,1,-554.416203,-9.709802e+05,-375.955729,0.979189,-519.991250,-6.446127e+05,-367.002604,0.985937,2.686673,0.009876,17.212476,163183.764543,4.476563,0.003374
0,ExtraTreesRegressor,4.159771,0.698113,-559.335085,-8.494249e+05,-388.580729,0.981660,NaN,16,{'n_estimators': 16},2,2,2,2,-571.279685,-9.974569e+05,-392.546875,0.978621,-547.390485,-7.013930e+05,-384.614583,0.984698,1.580818,0.015755,11.944600,148031.968874,3.966146,0.003039
1,RandomForestRegressor,20.596665,1.840577,-569.486916,-9.106514e+05,-389.895833,0.980339,NaN,32,{'n_estimators': 32},1,1,1,1,-586.940527,-1.074375e+06,-396.973958,0.976972,-552.033306,-7.469279e+05,-382.817708,0.983705,6.042391,0.999129,17.453611,163723.442741,7.078125,0.003366
0,RandomForestRegressor,12.782685,0.623131,-583.344186,-9.467510e+05,-399.158854,0.979562,NaN,16,{'n_estimators': 16},2,2,2,2,-603.472702,-1.132387e+06,-403.005208,0.975729,-563.215670,-7.611150e+05,-395.312500,0.983395,5.334297,0.086324,20.128516,185635.915032,3.846354,0.003833
1,GradientBoostingRegressor,4.165983,0.368742,-722.855038,-1.227826e+06,-547.791066,0.973467,0.8,32,"{'learning_rate': 0.8, 'n_estimators': 32}",1,1,1,1,-706.555655,-1.322788e+06,-532.700853,0.971648,-739.154420,-1.132865e+06,-562.881280,0.975285,1.983067,0.076971,16.299383,94961.226862,15.090213,0.001818
3,GradientBoostingRegressor,3.560088,0.173555,-795.338500,-1.447054e+06,-618.549952,0.968725,1,32,"{'learning_rate': 1.0, 'n_estimators': 32}",2,2,2,2,-773.410486,-1.536364e+06,-591.534346,0.967070,-817.266513,-1.357745e+06,-645.565559,0.970379,1.050746,0.048340,21.928013,89309.221054,27.015606,0.001654
0,GradientBoostingRegressor,2.112136,0.240463,-855.728778,-1.524914e+06,-665.283906,0.967042,0.8,16,"{'learning_rate': 0.8, 'n_estimators': 16}",3,3,3,3,-840.195331,-1.616929e+06,-653.833435,0.965344,-871.262225,-1.432898e+06,-676.734376,0.968739,1.112064,0.009009,15.533447,92015.422916,11.450471,0.001698
2,GradientBoostingRegressor,2.431509,0.304080,-935.774290,-1.767712e+06,-739.268400,0.961773,1,16,"{'learning_rate': 1.0, 'n_estimators': 16}",4,4,4,4,-894.165004,-1.765021e+06,-694.091331,0.962170,-977.383576,-1.770404e+06,-784.445470,0.961376,0.892458,0.056391,41.609286,2691.792998,45.177069,0.000397
0,AdaBoostRegressor,4.942620,0.534743,-1754.781104,-5.265357e+06,-1415.406795,0.886147,NaN,16,{'n_estimators': 16},1,1,1,1,-1734.448751,-5.318637e+06,-1372.279317,0.886003,-1775.113458,-5.212078e+06,-1458.534274,0.886291,1.691363,0.336638,20.332354,53279.912668,43.127479,0.000144
1,AdaBoostRegressor,6.767964,0.447073,-1902.206799,-6.393118e+06,-1499.619024,0.861699,NaN,32,{'n_estimators': 32},2,2,2,2,-1879.555675,-6.130318e+06,-1512.437932,0.868606,-1924.857922,-6.655918e+06,-1486.800116,0.854792,1.215977,0.001895,22.651123,262800.154642,12.818908,0.006907


In [262]:
for dat in [demand, offshore, onshore, solar]:
    X = dat.drop(columns='value')
    y = dat.value
    helper = EstimatorSelectionHelper(models, params, scoring=['neg_mean_absolute_error','neg_mean_squared_error','neg_median_absolute_error','r2'])
    helper.fit(X, y, n_jobs=2)
    display(helper.score_summary())

Running GridSearchCV for ExtraTreesRegressor.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:   21.1s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Running GridSearchCV for RandomForestRegressor.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:   37.2s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Running GridSearchCV for AdaBoostRegressor.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


...

Running GridSearchCV for AdaBoostRegressor.
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=2)]: Done   6 out of   6 | elapsed:   53.7s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


Running GridSearchCV for GradientBoostingRegressor.
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=2)]: Done  12 out of  12 | elapsed:   37.5s finished


,estimator,mean_fit_time,mean_score_time,mean_test_neg_mean_absolute_error,mean_test_neg_mean_squared_error,mean_test_neg_median_absolute_error,mean_test_r2,param_learning_rate,param_n_estimators,params,rank_test_neg_mean_absolute_error,rank_test_neg_mean_squared_error,rank_test_neg_median_absolute_error,rank_test_r2,split0_test_neg_mean_absolute_error,split0_test_neg_mean_squared_error,split0_test_neg_median_absolute_error,split0_test_r2,split1_test_neg_mean_absolute_error,split1_test_neg_mean_squared_error,split1_test_neg_median_absolute_error,split1_test_r2,split2_test_neg_mean_absolute_error,split2_test_neg_mean_squared_error,split2_test_neg_median_absolute_error,split2_test_r2,std_fit_time,std_score_time,std_test_neg_mean_absolute_error,std_test_neg_mean_squared_error,std_test_neg_median_absolute_error,std_test_r2
0,ExtraTreesRegressor,15.708273,3.390402,-0.003848,-0.000058,-0.000208,0.997822,NaN,16,{'n_estimators': 16},2,2,2,2,-0.004047,-0.000063,-0.000223,0.997597,-0.003810,-0.000057,-0.000194,0.997836,-0.003686,-0.000053,-0.000207,0.998032,6.448401,1.011202,0.000150,0.000004,0.000012,0.000178
1,ExtraTreesRegressor,14.409080,2.437239,-0.003738,-0.000055,-0.000202,0.997930,NaN,32,{'n_estimators': 32},1,1,1,1,-0.003894,-0.000059,-0.000209,0.997754,-0.003716,-0.000054,-0.000197,0.997928,-0.003603,-0.000051,-0.000200,0.998108,4.630697,0.369062,0.000120,0.000003,0.000005,0.000145
0,RandomForestRegressor,12.439532,0.984190,-0.003903,-0.000059,-0.000218,0.997755,NaN,16,{'n_estimators': 16},2,2,2,2,-0.004085,-0.000065,-0.000231,0.997518,-0.003891,-0.000059,-0.000206,0.997759,-0.003731,-0.000054,-0.000217,0.997987,5.668179,0.159709,0.000145,0.000005,0.000010,0.000191
1,RandomForestRegressor,25.017198,1.634999,-0.003820,-0.000057,-0.000216,0.997846,NaN,32,{'n_estimators': 32},1,1,1,1,-0.004008,-0.000063,-0.000230,0.997616,-0.003783,-0.000056,-0.000209,0.997870,-0.003668,-0.000052,-0.000208,0.998052,11.497519,0.190096,0.000142,0.000004,0.000010,0.000179
0,AdaBoostRegressor,11.261415,0.586122,-0.017123,-0.000631,-0.006607,0.976098,NaN,16,{'n_estimators': 16},1,2,1,2,-0.017579,-0.000598,-0.008377,0.977264,-0.017844,-0.000674,-0.007342,0.974281,-0.015948,-0.000622,-0.004102,0.976748,5.408045,0.020432,0.000838,0.000032,0.001821,0.001302
1,AdaBoostRegressor,16.493768,1.000921,-0.020915,-0.000620,-0.015672,0.976549,NaN,32,{'n_estimators': 32},2,1,2,1,-0.020897,-0.000602,-0.016420,0.977099,-0.020309,-0.000608,-0.014062,0.976810,-0.021541,-0.000649,-0.016534,0.975738,4.963543,0.167010,0.000503,0.000021,0.001139,0.000585
0,GradientBoostingRegressor,3.133151,0.314721,-0.009327,-0.000280,-0.001054,0.989401,0.8,16,"{'learning_rate': 0.8, 'n_estimators': 16}",4,4,1,4,-0.009075,-0.000261,-0.001051,0.990066,-0.009752,-0.000296,-0.001333,0.988723,-0.009154,-0.000283,-0.000777,0.989413,1.549833,0.007532,0.000302,0.000014,0.000227,0.000548
1,GradientBoostingRegressor,6.339025,0.387064,-0.007448,-0.000182,-0.001552,0.993116,0.8,32,"{'learning_rate': 0.8, 'n_estimators': 32}",2,2,4,2,-0.007414,-0.000185,-0.001801,0.992977,-0.007520,-0.000184,-0.001546,0.992981,-0.007410,-0.000177,-0.001308,0.993389,3.274190,0.004905,0.000051,0.000004,0.000202,0.000194
2,GradientBoostingRegressor,3.544334,0.324653,-0.008641,-0.000245,-0.001465,0.990742,1,16,"{'learning_rate': 1.0, 'n_estimators': 16}",3,3,3,3,-0.008594,-0.000238,-0.002075,0.990943,-0.008472,-0.000242,-0.001053,0.990782,-0.008857,-0.000254,-0.001266,0.990501,1.825269,0.005316,0.000161,0.000007,0.000440,0.000182
3,GradientBoostingRegressor,5.684604,0.405468,-0.007169,-0.000175,-0.001379,0.993376,1,32,"{'learning_rate': 1.0, 'n_estimators': 32}",1,1,2,1,-0.006927,-0.000167,-0.001396,0.993652,-0.007172,-0.000172,-0.001477,0.993423,-0.007409,-0.000186,-0.001265,0.993052,2.210399,0.086747,0.000197,0.000008,0.000087,0.000247
